In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv(r"..\preprocessing\county_variables.csv")
num_lag = 5 # number of lag features to predict from

data_cols = [col for col in data.columns if col != 'Date']
for lag in range(1, num_lag+1):
    for col in data_cols:
        data[f'{col}_lag{num_lag}'] = data[col].shift(lag)

num_targets = 1 # number of forward features to predict
for target in range(1, num_targets+1):
    for col in data_cols:
        data[f'{col}_target{target}'] = data[col].shift(-target)

# time input features
data['Date'] = pd.to_datetime(data['Date'])
data['doy'] = data['Date'].dt.day_of_year
data['cos_doy'] = np.cos(2*np.pi*data['Date'].dt.day_of_year/365.25)
data['sin_doy'] = np.sin(2*np.pi*data['Date'].dt.day_of_year/365.25)

data = data.iloc[num_lag:-num_targets]

total_data_points = len(data.index)
training_data_pct = 0.8
training_data = data.iloc[:int(training_data_pct * total_data_points)]
testing_data = data.iloc[int(training_data_pct * total_data_points):]

training_data.to_csv('training.csv', index=False)
testing_data.to_csv('testing.csv', index=False)

In [ ]:
input_features = [col for col in training_data.columns if ('target' not in col and col != 'Date')]
output_features = [col for col in training_data.columns if ('target' in col and col != 'Date')]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ReLU
from tensorflow.keras.activations import sigmoid, relu, linear, leaky_relu
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredError

In [ ]:
# architecture of deep neural network
model = Sequential(name='WeatherModel')

model.add(Input((len(input_features),)))

layer_sizes = [512, 784, 784, 512, 256]
#activation = leaky_relu

for layer_size in layer_sizes:
    model.add(Dense(layer_size))
    model.add(ReLU(negative_slope=0.5))

model.add(Dense(len(output_features)))
model.add(ReLU(negative_slope=0.5))

#model.summary()
model.compile(loss='mse', optimizer='adam')#, metrics = ['root_mean_squared_error'])

In [ ]:
model.fit(training_data[input_features], training_data[output_features], epochs=500, batch_size=30)

In [ ]:
model.evaluate(testing_data[input_features], testing_data[output_features])